## Acquire Function

>Importing host, user, and password from env module<br>
>Importing os and pandas modules

In [1]:
from env import host, user, password
import pandas as pd
import os

>url string that will be used to connect to db

In [2]:
database = 'mall_customers'
url = f'mysql+pymysql://{user}:{password}@{host}/{database}'

>Reading SQL query into a df using the query string, url string, and `customer_id` as the index column

In [3]:
query = 'select * from customers'
mall_df = pd.read_sql(query, url, index_col = 'customer_id')

In [4]:
mall_df.head()

,gender,age,annual_income,spending_score
customer_id,,,,
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40


>## `acquire_data()` function

In [14]:
def acquire_data(file_name, index_col, database, query):
    '''
THIS FUNCTION TAKES IN:
    (1) A CSV FILE NAME ***INCLUDE .CSV EXT****, 
    (2) COLUMN FROM TABLE TO BE USED AS DATAFRAME INDEX, 
    (3) DATABASE NAME, AND 
    (4) SQL QUERY STRING 
AND RETURNS A PANDAS DF BY:
        (i.) CHECKING TO SEE IF LOCAL CSV FILE WITH DATA EXISTS
        (ii.) WRITING THE LOCAL CSV FILE OT PANDAS DF 
OR, IF LOCAL CSV DOES NOT EXIST:
        (i.) IMPORTING DATABASE CONNECTION CREDENTIALS
        (ii.) USING CREDENTIALS TO CREATE DATABASE CONNECTION STRING
        (iii.) READING THE SQL QUERY INTO A DF
        (iv.) CACHING DF AND SAVING DATA AS LOCAL CSV FILE
    '''
    # checking for local csv file
    if os.path.isfile(file_name):
        
        # reading csv file to pandas df
        df = pd.read_csv(file_name, index_col = index_col)
        
    # if no local csv file
    else:

        # url database connection string
        url = f'mysql+pymysql://{user}:{password}@{host}/{database}'

        # reading sql query into df using query, url string, and index_col parameter
        df = pd.read_sql(query, url, index_col)
        
        # write df to local csv file
        df.to_csv(file_name)
    
    return df    

In [15]:
test_df = acquire_data('mall_data.csv', 'customer_id', 'mall_customers', 'select * from customers')

In [16]:
test_df.head()

,gender,age,annual_income,spending_score
customer_id,,,,
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40


## Summarize Function

In [25]:
# 1) shape
print('DataFrame Shape')
print('--------------------')
print(f'Rows: {mall_df.shape[0]}\nColumns: {mall_df.shape[1]}')
print()

DataFrame Shape
--------------------
Rows: 200
Columns: 4



>200 observations and 4 variables

In [21]:
# 2) info
print('DataFrame Info')
print('--------------------')
mall_df.info()
print()

DataFrame Info
--------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gender          200 non-null    object
 1   age             200 non-null    int64 
 2   annual_income   200 non-null    int64 
 3   spending_score  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 7.8+ KB



>No null values<br>
x3 variables with int dtype and x1 as object

In [28]:
# 3) descriptive stats

print('DataFrame Stats')
print('--------------------')
mall_df.describe()

DataFrame Stats
--------------------


,age,annual_income,spending_score
count,200.00,200.00,200.00
mean,38.85,60.56,50.20
std,13.97,26.26,25.82
min,18.00,15.00,1.00
25%,28.75,41.50,34.75
50%,36.00,61.50,50.00
75%,49.00,78.00,73.00
max,70.00,137.00,99.00


>will need to scale numerical columns for modeling (different measurement units)

In [44]:
# 4) value counts for colums with 10 or less values

for col in mall_df.columns:
    if mall_df[col].nunique() <= 10:
        print(col.upper())
        print('-'*len(col))
        print(mall_df[col].value_counts())
        print('_____________________________')
        print()
    else:
        print(col.upper())
        print('-'*len(col))
        print('This column has > 10 values')
        print('_____________________________')
        print()

GENDER
------
Female    112
Male       88
Name: gender, dtype: int64
_____________________________

AGE
---
This column has > 10 values
_____________________________

ANNUAL_INCOME
-------------
This column has > 10 values
_____________________________

SPENDING_SCORE
--------------
This column has > 10 values
_____________________________



>will need to encode gender (binary variable)

#### Even though there are no nulls in this data, I want to create a function that can summarize and handle nulls by rows and columns

In [46]:
# 5a) Nulls by column
# number of nulls in each column
mall_df.isnull().sum()

gender            0
age               0
annual_income     0
spending_score    0
dtype: int64

In [54]:
# percentage of nulls in each column
mall_df.isnull().sum()/len(mall_df)*100

gender                           0.00
age                              0.00
annual_income                    0.00
spending_score                   0.00
dtype: float64

In [49]:
# 5b) Nulls by row
mall_df.isnull().sum(axis = 1)

customer_id
1      0
2      0
3      0
4      0
5      0
      ..
196    0
197    0
198    0
199    0
200    0
Length: 200, dtype: int64

In [50]:
mall_df.isnull().sum(axis = 1).value_counts()

0    200
dtype: int64

## Prepare Function

>Outliers

In [ ]:
# def remove_outliers(df, outlier_cols, k):
    
#     for col in df[outlier_cols]:
#         q1 = df[col].quantile(.25)
#         q3 = df[col].quantile(.75)
        
#         iqr = q3 - q1
        
#         upper_bound = q3 + k * iqr
#         lower_bound = q1 - k * iqr
        
#         df = df[(df[col] < upper_bound) & (df[col] > lower_bound)]
        
#     return df

In [ ]:
# mall_df_outliers = remove_outliers(mall_df, ['age', 'spending_score', 'annual_income'], 50)
# mall_df_outliers.head()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# for col in mall_df.columns:
#     if mall_df[col].dtype != 'O':
#         sns.boxplot(y = mall_df[col], data = mall_df)
#         plt.title(col)
#         plt.show()
#         print('-------------------')
#         print()

In [ ]:
# for i, col in enumerate(mall_df.columns):
    
#     if mall_df[col].dtype != 'O':
        
#         plot_number = i + 1

#         plt.subplot(1, len(mall_df.columns), plot_number)

#         plt.title(f'{col}')

#         sns.boxplot(data = mall_df[[col]])
        
#         plt.tight_layout(pad = 2)

# print('Boxplot Distributions of Numerical Variables With Outliers')

   
    

In [ ]:
# for i, col in enumerate(mall_df_outliers.columns):
    
#     if mall_df_outliers[col].dtype != 'O':
        
#         plot_number = i + 1

#         plt.subplot(1, len(mall_df_outliers.columns), plot_number)

#         plt.title(f'{col}')

#         sns.boxplot(data = mall_df_outliers[[col]])
        
#         plt.tight_layout(pad = 2)
        
# print('Boxplot Distributions of Numerical Variables Without Outliers')

In [ ]:
# mall_df.hist()
# plt.tight_layout(pad = 2)

# mall_df_outliers.hist()
# plt.tight_layout(pad = 2)
